In [1]:
## Imports

import sys, os
from pathlib import Path

parent_folder = str(Path.cwd().parents[1])
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from sigpy import mri
import scipy
import pickle
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
import seaborn as sns
import sigpy as sp
import cupy as cp
import numpy as np
from sigpy.mri.app import TotalVariationRecon, L1WaveletRecon
from scipy.io import savemat
import twixtools
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from scipy.signal import butter,filtfilt


## My files
from ksp_plot_helpers import plot_ksp_data_multichannel, find_and_plot_acquired_region
from raw_data_utils import get_kspace_data, get_TR
from resp_signal_functions import resp_signal_all_slices, resp_signal_single_slice, resp_signal_center_sample_single_slice
from resp_signal_plot_functions import *
import gating_functions
from pca_helper import pca_resp_signal

In [2]:
data_file_pt2 = '/data/lilianae/NaF_Patient2/anon_meas_MID00082_FID64646_Tho_fl3d_star_vibe_991_nav_tj_2000sp_AllCoils_SOS_2.dat'
# data_file_pt1 = '/data/lilianae/NaF_MtSinai/anon_meas_MID00118_FID60738_Tho_fl3d_star_vibe_991_nav_tj_2000sp_AllCoils_SOS.dat'


multi_twix = twixtools.read_twix(str(data_file_pt2))
mapped = twixtools.map_twix(multi_twix)

# # mapped[0] is sens data
# data_0 = mapped[0]['image']
# data_0.flags['remove_os']=True
# echo_num=0                                                 # first echo is spoke data
# num_points = int(mapped[0]['hdr']['Config']['NImageLins'])  # number of points on one spoke
# num_full_par = int(mapped[0]['hdr']['Config']['NImagePar'])  # number of points on one spoke)
# print(f'Full number of partitions = {num_full_par}')
# print(f'Number of readouts = {num_points}')


# ksp_data = data_0[...,echo_num,0,0,0,:,0,0,:,:,:]
# ksp_data = ksp_data.squeeze()
# # print(ksp_data.shape)
# ksp_data = np.transpose(ksp_data,(2,0,1,3))
# print(f'ksp_data.shape = {ksp_data.shape}')  ## Shape = (15, 58, 2002, 256) -> (channels, partitions, lines, columns)
    

Software version: VD/VE (!?)

Scan  0


100%|██████████| 15.9G/15.9G [00:12<00:00, 1.32GB/s]


In [3]:
from twixtools.recon_helpers import remove_oversampling
chronological_data = []
mdb_list = []

for i, mdb in enumerate(multi_twix[-1]['mdb']):
## Use same logic as twix_category['image'] to get mdh values for k-space
    if (not mdb.is_flag_set('SYNCDATA') and
        not mdb.is_flag_set('ACQEND') and
        not mdb.is_flag_set('RTFEEDBACK') and
        not mdb.is_flag_set('HPFEEDBACK') and
        not mdb.is_flag_set('REFPHASESTABSCAN') and
        not mdb.is_flag_set('PHASESTABSCAN') and
        not mdb.is_flag_set('PHASCOR') and
        not mdb.is_flag_set('NOISEADJSCAN') and
        not mdb.is_flag_set('noname60') and
        (not mdb.is_flag_set('PATREFSCAN') or mdb.is_flag_set('PATREFANDIMASCAN'))):

        if not np.isnan(mdb.mdh.TimeStamp):
            ## Extract k-space data for this readout
            mdb_data = mdb.data  # Shape: (channels, samples)

            ## Apply oversampling removal to ensure consistent array sizes
            if mdb_data.shape[-1] == 512:  ## if we have 512 points, this is an image line. If there are 704 points, it is a noise scan. Discovered from manual inspection

                # mdb_data, _ = remove_oversampling(mdb_data, x_was_in_timedomain=True)
                mdb_data = mdb_data    ## Only take first 256 samples, same as logic Michael used in former data processing code
                mdb_list.append(mdb)
                
                
                chronological_data.append({
                    'timestamp': mdb.mdh.TimeStamp,
                    'partition': mdb.mdh.Counter.Par,
                    'line': mdb.mdh.Counter.Lin,
                    'kspace_data': mdb_data,  # Shape: (channels, 256)
                    'ice_param' : mdb.mdh.IceProgramPara[2],
                    'acquisition_index': i  # Original position in MDB list
                })

In [4]:
import save_data_helpers

save_data_helpers.write_pickle(mdb_list, 'full_mdb_list_with_os.pkl')

TypeError: cannot pickle '_io.BufferedReader' object

In [ ]:
# print(len(mdb_list))

In [ ]:
# print(mdb_list[0].shape)

In [ ]:
# import save_data_helpers

# save_data_helpers.write_pickle(mdb_list, 'mdb_list_with_os.pkl')

In [ ]:
import save_data_helpers

mdb_list = save_data_helpers.read_pickle('mdb_list_with_os.pkl')

In [5]:
# read image data from list of mdbs and sort into 3d k-space (+ coil dim.)
def import_kspace(image_mdbs):

    n_line = 1 + max([mdb.cLin for mdb in image_mdbs])
    n_part = 1 + max([mdb.cPar for mdb in image_mdbs])
    n_channel, n_column = image_mdbs[0].data.shape

    out = np.zeros([n_part, n_line, n_channel, n_column], dtype=np.complex64)
    for mdb in image_mdbs:
        # '+=' takes care of averaging, but careful in case of other counters (e.g. echoes)
        out[mdb.cPar, mdb.cLin] += mdb.data

    return out  # 4D numpy array [n_part, n_line, n_channel, n_column]

In [6]:
out = import_kspace(mdb_list)
print(f'out.shape = {out.shape}')

out.shape = (58, 2002, 15, 512)


In [7]:
save_data_helpers.write_pickle(out, 'ksp_from_mdb_512_samples.pkl')

Successfully saved as ksp_from_mdb_512_samples.pkl
